In [1]:
import pandas as pd
import os
import keras

from tensorflow.keras.applications import VGG16
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

/var/folders/my/rbl5q7yd7pxb2yk9_qjc7mjr0000gn/T/ipykernel_65125/952801664.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('list_attribute.txt', delim_whitespace=True, skiprows=1)

file_images = os.listdir('images/')
df_filtered = df[df.index.isin(file_images)]

df_filtered.describe()

/var/folders/my/rbl5q7yd7pxb2yk9_qjc7mjr0000gn/T/ipykernel_65125/2892880123.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('list_attribute.txt', delim_whitespace=True, skiprows=1)


,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
count,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.0000
mean,-0.785200,-0.46920,0.031200,-0.591200,-0.948800,-0.706000,-0.542800,-0.548800,-0.532800,-0.717600,...,-0.887600,-0.056000,-0.588000,-0.388400,-0.628400,-0.903200,-0.060400,-0.75800,-0.858800,0.5552
std,0.619304,0.88318,0.999613,0.806606,0.315909,0.708283,0.839946,0.836037,0.846326,0.696525,...,0.460661,0.998531,0.808942,0.921583,0.777968,0.429263,0.998274,0.65232,0.512362,0.8318
min,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.0000
25%,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,1.0000
50%,-1.000000,-1.00000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,1.0000
75%,-1.000000,1.00000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,1.000000,-1.000000,1.000000,-1.000000,-1.000000,1.000000,-1.00000,-1.000000,1.0000
max,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.0000


In [3]:
df_filtered.reset_index(inplace=True)

df_compiled = df_filtered[['index', 'Male']]

df_compiled

,index,Male
0,000051.jpg,1
1,000052.jpg,1
2,000065.jpg,1
3,000166.jpg,1
4,000198.jpg,-1
...,...,...
4995,202320.jpg,-1
4996,202340.jpg,-1
4997,202347.jpg,-1
4998,202357.jpg,-1


In [4]:
df_compiled[df_compiled['Male'] == -1] = 0
df_compiled['Male'] = df_compiled['Male'].astype('string')

/var/folders/my/rbl5q7yd7pxb2yk9_qjc7mjr0000gn/T/ipykernel_65125/724312737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compiled[df_compiled['Male'] == -1] = 0
/var/folders/my/rbl5q7yd7pxb2yk9_qjc7mjr0000gn/T/ipykernel_65125/724312737.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compiled['Male'] = df_compiled['Male'].astype('string')


In [5]:
df_compiled.loc[df_compiled['Male'] == '1', ['Male']] = 'Male'
df_compiled.loc[df_compiled['Male'] == '0', ['Male']] = 'Female'
df_compiled

,index,Male
0,000051.jpg,Male
1,000052.jpg,Male
2,000065.jpg,Male
3,000166.jpg,Male
4,0,Female
...,...,...
4995,0,Female
4996,0,Female
4997,0,Female
4998,0,Female


In [6]:
train_files = os.listdir('train/')
test_files = os.listdir('test/')
df_train = df_compiled[df_compiled['index'].isin(train_files)]
df_test = df_compiled[df_compiled['index'].isin(test_files)]


In [7]:
IMG_SIZE = (218, 178)
BATCH_SIZE = 30
EPOCH_COUNT = 20

In [8]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    './train',
    x_col='index',
    y_col='Male',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 1434 validated image filenames belonging to 1 classes.


In [9]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    df_test,
    './test',
    x_col='index',
    y_col='Male',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 613 validated image filenames belonging to 1 classes.


In [10]:
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape= IMG_SIZE + (3,))

base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 218, 178, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 218, 178, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 218, 178, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 109, 89, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 109, 89, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 109, 89, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 54, 44, 128)       0     

In [11]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 5, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 15360)             0         
                                                                 
 dense (Dense)               (None, 256)               3932416   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 18647361 (71.13 MB)
Trainable params: 3932673 (15.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [13]:
from keras.optimizers import Adam

adam = Adam(learning_rate=0.01)

In [14]:
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint

earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=4,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.01)
# Define a ModelCheckpoint callback to save the model weights during training
checkpoint_callback = ModelCheckpoint(filepath='Dataset/vgg16_weights_lr01.h5',
                                      save_best_only=True,
                                      save_weights_only=False)

callbacks = [checkpoint_callback, earlystop, learning_rate_reduction]


In [16]:

history = model.fit(train_generator,
                    epochs=EPOCH_COUNT,
                    validation_data=val_generator,
                    validation_steps=len(df_test),
                    steps_per_epoch=len(df_train),
                    verbose = 1,
                    callbacks=callbacks
                    )


Epoch 1/20
  38/1434 [..............................] - ETA: 43:13 - loss: 0.0413 - accuracy: 0.9735